In [54]:
import yagmail
import pandas as pd
from funciones_recommendation_system import get_datasets, match_researcher_call, recommendation_system_researcher_call, match_call_researcher, recommendation_system_call_researcher

In [63]:
# load databases
path = '/export/data_ml4ds/AI4U/Datasets/'
version_wp = '20240510'
version_rp = '20240321'
df_publications, df_projects, df_publications_researchers,df_projects_researchers, df_researchers, df_calls = get_datasets(path, version_wp, version_rp)

# load the validation set
df_val = pd.read_excel('/export/usuarios_ml4ds/mafuello/Github/recommendation_system_validation/validation_set.xlsx')
df_val = df_val.rename(columns={'research portal': 'invID'})
df_val = df_val.rename(columns={'invID': 'id_RP'}).merge(df_researchers[['id_RP', 'id_researcher']], on='id_RP')
df_val = df_val.drop_duplicates('id_RP')

/home/mbalairon/.local/lib/python3.10/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


In [ ]:
def send_mail(origin_mail, password, mail_adress, subject, content):
    '''
    Function for sending mails automatically
    
    origin_mail -> Your e-mail adress
    password -> application password 
    mail_adress -> Recipient e-mail adress
    subject -> e-mail subject
    content -> e-mail content
    '''
    
    yag = yagmail.SMTP(origin_mail, password)
    try:
        yag.send(to=mail_adress, subject=subject, contents=content)
        print("Correo enviado exitosamente a ", mail_adress)
    except Exception as e:
        print(f"Error al enviar el correo: {e}")

In [58]:
df_eic = pd.read_parquet(path + 'work_programmes/20231001/EIC_work_programmes.parquet')
df_horizon = pd.read_parquet(path +'work_programmes/{}/horizon_work_programmes.parquet'.format(version_wp))

In [65]:
df_researchers['id_researcher']

0          0
1          1
2          2
3          3
4          4
        ... 
1434    1434
1435    1435
1436    1436
1437    1437
1438    1438
Name: id_researcher, Length: 1439, dtype: int64

In [53]:
origin_mail = 'mbalairo@pa.uc3m.es'
password = 'ztbm nisu cats rnjk'
subject = 'Pruebas 2'

for i in df_val.head(1).index:
    invID = df_val['id_researcher'][i]
    name = df_val['Nombre y Apellidos IP'][i]
    mail_adress = 'malmagro@pa.uc3m.es'
    #mail_adress = df_val['email'][i]

        
    ranking_words = recommendation_system_researcher_call(method='BERT', agg_method='mean_imp', researcher=invID, calls=df_calls, n=3)
    ranking_words = ranking_words[['Call']].merge(df_horizon, on='Call')[['Call', 'Work Programme', 'Title', 'Expected Outcome']]

    ranking_topics = recommendation_system_researcher_call(method='separated', agg_method='mean_imp', researcher=invID, calls=df_calls, n=3)
    ranking_topics = ranking_topics[['Call']].merge(df_horizon, on='Call')[['Call', 'Work Programme', 'Title', 'Expected Outcome']]    
    
    # Convertir ranking_words a una tabla HTML
    ranking_words_html = """
    <table style="width: 100%; border-collapse: collapse; border: 1px solid black;">
        <tr>
            <th style="border: 1px solid black; padding: 8px; text-align: center;">ID</th>
            <th style="border: 1px solid black; padding: 8px; text-align: center;">Título</th>
            <th style="border: 1px solid black; padding: 8px; text-align: center;">Cluster</th>
            <th style="border: 1px solid black; padding: 8px; text-align: center;">Expected Outcome</th>
        </tr>
    """
    for idx, row in ranking_words.iterrows():
        ranking_words_html += f"""
        <tr>
            <td style="border: 1px solid black; padding: 8px; text-align: center;">{row['Call']}</td>
            <td style="border: 1px solid black; padding: 8px;">{row['Title']}</td>
            <td style="border: 1px solid black; padding: 8px; text-align: center;">{row['Work Programme']}</td>
            <td style="border: 1px solid black; padding: 8px;">{row['Expected Outcome']}</td>
        </tr>
        """
    ranking_words_html += "</table>"

    # Convertir ranking_topics a una tabla HTML
    ranking_topics_html = """
    <table style="width: 100%; border-collapse: collapse; border: 1px solid black;">
        <tr>
            <th style="border: 1px solid black; padding: 8px; text-align: center;">ID</th>
            <th style="border: 1px solid black; padding: 8px; text-align: center;">Título</th>
            <th style="border: 1px solid black; padding: 8px; text-align: center;">Cluster</th>
            <th style="border: 1px solid black; padding: 8px; text-align: center;">Expected Outcome</th>
        </tr>
    """
    for idx, row in ranking_topics.iterrows():
        ranking_topics_html += f"""
        <tr>
            <td style="border: 1px solid black; padding: 8px; text-align: center;">{row['Call']}</td>
            <td style="border: 1px solid black; padding: 8px;">{row['Title']}</td>
            <td style="border: 1px solid black; padding: 8px; text-align: center;">{row['Work Programme']}</td>
            <td style="border: 1px solid black; padding: 8px;">{row['Expected Outcome']}</td>
        </tr>
        """
    ranking_topics_html += "</table>"
    
    content = f"""
    <html>
    <body style="font-family: Arial, sans-serif;">
        <p style="margin-bottom: 0.5px;">Buenos días {name},</p>
        <p style="margin-bottom: 0.5px;">INTRODUCCIÓN:</p>
        <p style="margin-bottom: 0.5px;">Estos son los resultados de las recomendaciones:</p>
        
        <strong><p style="margin-bottom: 0.5px;">Ranking basado en las palabras de sus publicaciones:</p></strong>
        {ranking_words_html}
        
        <strong><p style="margin-bottom: 0.5px;">Ranking basado en las temáticas de sus publicaciones:</p></strong>
        {ranking_topics_html}
        
        <p style="margin-top: 0.5px;">Saludos,</p>
        <p>Equipo de Investigación</p>
    </body>
    </html>
    """
    
    send_mail(origin_mail, password, mail_adress, subject, content)

Correo enviado exitosamente a  malmagro@pa.uc3m.es
